# Session 6: AI API 호출(Inference) 및 프로젝트 완성

### ■ Session Objective
-   Session 5에서 배포한 `loan-approval-service` Endpoint에 **Vertex AI SDK**를 사용하여 실시간 예측 요청을 전송합니다.
-   가상의 대출 신청자 데이터를 API가 요구하는 형식(JSON)으b로 변환하고, API의 응답(승인/거절)을 확인하여 1차시 프로젝트를 완성합니다.
-   실습 완료 후, **엔드포인트 배포를 취소(Undeploy)하여 불필요한 비용 발생을 중단**합니다.

---

### ■ Key Concepts

#### 1️⃣ Vertex AI SDK (Software Development Kit)
- Python 코드에서 GCP 서비스(Vertex AI, GCS 등)와 상호작용할 수 있도록 Google이 제공하는 공식 라이브러리입니다.  
- 복잡한 HTTP 요청을 직접 작성할 필요 없이,  
  `endpoint.predict()` 같은 간단한 함수 호출로 Vertex AI API를 사용할 수 있습니다.

---

#### 2️⃣ API 요청 페이로드 (Request Payload) - 수정된 부분
- Vertex AI Endpoint는 예측 요청 시, 내부적으로 **JSON 형식의 `{"instances": [...]}`** 구조를 사용합니다.  
- 하지만 **Python SDK(`aiplatform.Endpoint.predict`)를 사용할 경우**,  
  사용자가 직접 JSON을 작성할 필요는 없습니다.  
  단순히 **리스트 형태의 데이터를 `instances` 인자로 전달**하면,  
  SDK가 자동으로 `{"instances": [...]}` 구조로 변환해 API 요청을 전송합니다.

✅ **예시**

```python
# 사용자는 단순히 Python 리스트만 넘기면 됨
instances = [[5000.0, 2000.0, 150.0, 360.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
prediction = endpoint.predict(instances=instances)
```

↳ SDK 내부적으로는 다음과 같은 JSON 요청을 자동으로 생성합니다:
```json
{
  "instances": [
    [5000.0, 2000.0, 150.0, 360.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
  ]
}
```

---

#### 3️⃣ 데이터 일관성 (Data Consistency) — (⭐ 매우 중요 ⭐)
- API로 전송하는 데이터는 **모델 학습 시 사용된 입력 데이터와 동일한 구조와 순서**를 가져야 합니다.  
- 즉, **피처 순서(`MODEL_FEATURES`)**, **인코딩 방식**, **데이터 타입(float 변환 등)** 이 100% 일치해야 합니다.  
- 이를 보장하기 위해 학습 시 함께 저장했던 **`model_features.pkl`** 파일을 다시 불러와 컬럼 순서를 맞춥니다.


### 🚨 (필수) 비용 관리: 실습 후 엔드포인트 정리
-   본 세션이 끝나면, **비용이 계속 청구되는 엔드포인트를 즉시 '배포 취소'해야 합니다.**
-   이 노트북의 **마지막 단계**에 배포 취소 가이드가 포함되어 있습니다.

In [25]:
print("hello world")

hello world


In [27]:
# ==============================================================================
# [실습 1] API 호출을 위한 환경 설정
# ==============================================================================
import pandas as pd
import joblib, pickle 
from google.cloud import aiplatform  # Vertex AI SDK
from google.cloud import storage      # GCS SDK (피처 목록 로드용)

# --- (1/4) GCP 환경 변수 설정 ---
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ 수강생 각자의 GCP 프로젝트 ID로 이 부분을 수정해야 합니다! ※
# ※ GCP 콘솔 상단에서 프로젝트 ID를 복사해오세요.            ※
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
#PROJECT_ID = "여러분의-GCP-PROJECT-ID"  # 예: "gcp-project-12345"
PROJECT_ID = "sigma-nimbus-475403-v9"
REGION = "asia-northeast3"             # 배포 리전

# --- (2/4) GCS 클라이언트 초기화 ---
# (Session 4에서 업로드한 'model_features.pkl'을 다운로드하기 위함)
gcs_client = storage.Client(project=PROJECT_ID)

# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ Session 4에서 사용한 버킷 이름과 피처 파일 경로를 정확히 입력합니다.
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
#BUCKET_NAME = "여러분의-GCS-버킷-이름" # 예: "gcp-project-12345-loan-data"
BUCKET_NAME = "my_instance-ysh2-banking-data"
FEATURES_GCS_PATH = "new_feat_2/model_features.pkl"

# GCS에서 피처 목록 파일 로드
bucket = gcs_client.bucket(BUCKET_NAME)
blob = bucket.blob(FEATURES_GCS_PATH)
blob.download_to_filename("model_features.pkl") # Workbench 로컬에 임시 다운로드

with open("model_features.pkl", "rb") as f :
    MODEL_FEATURES = pickle.load(f)
print(f"▶ GCS에서 {len(MODEL_FEATURES)}개의 피처 목록을 성공적으로 로드했습니다.")
# print(MODEL_FEATURES) # (확인용)

# --- (3/4) Vertex AI SDK 초기화 ---
aiplatform.init(project=PROJECT_ID, location=REGION)

# --- (4/4) 엔드포인트 객체 가져오기 ---
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ GCP 콘솔 'Vertex AI > 엔드포인트' 메뉴에서 
# ※ '엔드포인트 ID' (긴 숫자)를 복사하여 붙여넣어야 합니다.
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
#ENDPOINT_ID = "여러분의-엔드포인트-ID" # 예: "1234567890123456789"
ENDPOINT_ID = "670264487315505152"

endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID)
print(f"▶ 엔드포인트(ID: {ENDPOINT_ID})에 성공적으로 연결되었습니다.")

▶ GCS에서 14개의 피처 목록을 성공적으로 로드했습니다.
▶ 엔드포인트(ID: 670264487315505152)에 성공적으로 연결되었습니다.


In [92]:
# ==============================================================================
# [실습 2] 예측 요청 전송 및 결과 확인
# ==============================================================================

# 1. 가상의 대출 신청자 데이터 (Raw Data)
#    (Session 3의 시뮬레이션 데이터와 동일)
sample_data = {
    'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Education': 'Graduate',
    'Self_Employed': 'No', 'ApplicantIncome': 5000, 'CoapplicantIncome': 2000,
    'LoanAmount': 150, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0,
    'Property_Area': 'Urban'
}
sample_df = pd.DataFrame([sample_data])
print(" [1. 원본 요청 데이터 (Raw Data)]")
sample_df.head()


 [1. 원본 요청 데이터 (Raw Data)]


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,Yes,1,Graduate,No,5000,2000,150,360.0,1.0,Urban


In [91]:
# -------------------------
# 2) 학습 시와 동일한 전처리 파이프라인 적용
#    - 여기서는 예시로 get_dummies + reindex만 표기
#    - 실제로는 학습 시 쓰던 전처리(스케일링, 인코딩 등)를 그대로 재현해야 함
# -------------------------

print("\n [2. 학습때와 동일한 데이터 전처리]")
sample_df_processed = pd.get_dummies(sample_df)
final_sample_df = sample_df_processed.reindex(columns=MODEL_FEATURES, fill_value=0) 
final_sample_df.head()


 [2. 학습때와 동일한 데이터 전처리]


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,5000,2000,150,360.0,1.0,True,True,True,0,0,0,0,0,True


In [100]:
# -------------------------
# 3) 타입 정규화 (bool/np.bool_ 포함 모든 값을 float로) 
# 수정된 부분
# -------------------------

final_sample_df = final_sample_df.astype(float)
final_sample_df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,5000.0,2000.0,150.0,360.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [102]:
# -------------------------
# 4) 리스트-오브-리스트 인스턴스 구성 (피처 순서 준수) -- 중요 -- 
# -------------------------
row = final_sample_df.iloc[0] 
instances = [row[MODEL_FEATURES].tolist()]  # [[x1, x2, ...]]

In [103]:
print(instances)

[[5000.0, 2000.0, 150.0, 360.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]


In [104]:
# API 호출을 통한 예측 결과 확인
prediction_result = endpoint.predict(instances=instances)


In [105]:
prediction_result

Prediction(predictions=[1.0], deployed_model_id='39778131669680128', metadata=None, model_version_id='1', model_resource_name='projects/672193963995/locations/asia-northeast3/models/2311907915352506368', explanations=None)

In [106]:
print("\n [5. 최종 예측 결과 해석]")
# 응답 형식: prediction_result.predictions[0]
final_prediction = prediction_result.predictions[0]

if final_prediction == 1:
    print(f"  ▶▶ 예측 결과: '승인 (Approved - Y)'")
else:
    print(f"  ▶▶ 예측 결과: '거절 (Refused - N)'")

print("\n==============================================================================")
print("▶▶▶ 1차시 프로젝트 API 구축 및 테스트가 성공적으로 완료되었습니다! ◀◀◀")
print("==============================================================================")


 [5. 최종 예측 결과 해석]
  ▶▶ 예측 결과: '승인 (Approved - Y)'

▶▶▶ 1차시 프로젝트 API 구축 및 테스트가 성공적으로 완료되었습니다! ◀◀◀


# [실습 3] (필수!) 엔드포인트 배포 취소 (비용 중단)

축하합니다! 여러분은 1차시의 모든 과정을 완료하고, 클라우드에 배포된 AI API를 호출하는 데 성공했습니다.

이제 **불필요한 비용이 계속 발생하는 것을 막기 위해** 배포된 엔드포인트 서비스를 즉시 중지(배포 취소)해야 합니다.

---

### ■ 배포 취소 절차 (GCP Console UI - 권장)

1.  GCP 콘솔에서 **'Vertex AI' > '엔드포인트(Endpoints)'** 메뉴로 이동합니다.
2.  우리가 생성한 **`loan-approval-service`** 엔드포인트의 이름(링크)을 클릭합니다.
3.  상세 정보 페이지 상단에서 **`배포된 모델 배포 취소`** (휴지통 아이콘 🗑️) 버튼을 클릭합니다.
4.  팝업창에서 배포 취소할 모델을 확인하고 `배포 취소` 버튼을 클릭합니다.
5.  **(확인)** 잠시 후, '배포된 모델' 섹션이 비어있게 되면 비용 청구가 중지된 것입니다. (엔드포인트 자체는 목록에 남아있어도 괜찮습니다.)

### ■ (선택) 엔드포인트 완전 삭제

-   배포 취소 후, 엔드포인트 목록에서 `loan-approval-service`를 체크하고 상단의 `삭제` 버튼을 누르면 목록에서도 완전히 사라집니다.

---
### ■ Session 6 및 1차시 교육 완료
수고하셨습니다. 이제 2차시 교육에서 이 API를 '전문가 Tool'로 활용할 준비가 모두 끝났습니다.